In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

In [3]:
def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

In [4]:
#Set target and input variables 
target_station = 'DHHL_3'

#All neighbor stations with residual correlation greater than .90
neighbor_stations_90 = ['DHHL_3',  'DHHL_4','DHHL_5','DHHL_10','DHHL_11','DHHL_9','DHHL_2', 'DHHL_6','DHHL_7','DHHL_8']

In [5]:
df = pd.read_pickle("df_oahu.pkl")
df_ssa_clean = pd.read_pickle("df_ssa_clean.pkl")
df_ssa_residual = pd.read_pickle("df_ssa_residual.pkl")

## Remove columns with many corrupted or missing values
df.drop(columns=['AP_1', 'AP_7'], inplace=True)
df_ssa_clean.drop(columns=['AP_1', 'AP_7'], inplace=True)
df_ssa_residual.drop(columns=['AP_1', 'AP_7'], inplace=True)

In [6]:
# Get data form the interval of interest
#interval = ((df.index >= '2010-06') & (df.index < '2010-08'))
interval = '2010-06'
df = df.loc[interval]
df_ssa_clean = df_ssa_clean.loc[interval]
df_ssa_residual = df_ssa_residual.loc[interval]

In [7]:
#Normalize Data

# Save Min-Max for Denorm
min_raw = df[target_station].min()
min_clean = df_ssa_clean[target_station].min()
min_residual = df_ssa_residual[target_station].min()

max_raw = df[target_station].max()
max_clean = df_ssa_clean[target_station].max()
max_residual = df_ssa_residual[target_station].max()

# Perform Normalization
norm_df = normalize(df)
norm_df_ssa_clean = normalize(df_ssa_clean)
norm_df_ssa_residual = normalize(df_ssa_residual)

## K-Means Clustering

In [8]:
from sklearn.cluster import MiniBatchKMeans

In [9]:
from ggplot import *

/Users/cseveriano/anaconda3/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/Users/cseveriano/anaconda3/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/Users/cseveriano/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [10]:
def kmeans_clustering(data, partitions, batch_size, init_size):
    clusterer = MiniBatchKMeans(init='k-means++', n_clusters=partitions, batch_size=batch_size, init_size=init_size,
                                        n_init=1, verbose=False)
    data_labels = clusterer.fit_predict(data)
    centroids = clusterer.cluster_centers_
    
    return data_labels, centroids

In [17]:
data = norm_df_ssa_clean[neighbor_stations_90].values

In [18]:
nclusters = 10

In [19]:
labels, centers = kmeans_clustering(data, nclusters, 100, 100)

### PCA Transformation

In [20]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(data)
pca_df = pd.DataFrame()

pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
#pca_df['pca-three'] = pca_result[:,2]
pca_df['label'] = labels

print ('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [  9.98322472e-01   1.28336550e-03   1.59814184e-04]


In [170]:
from ggplot import *

chart = ggplot( pca_df, aes(x='pca-one', y='pca-two', color='label') ) \
        + geom_point(size=75,alpha=0.8) \
        + ggtitle("First and Second Principal Components colored by digit")
chart

<ggplot: (204548398463)>

### TSNE Transformation

In [68]:
import time

from sklearn.manifold import TSNE

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 1830
[t-SNE] Computed conditional probabilities for sample 1830 / 1830
[t-SNE] Mean sigma: 0.000152
[t-SNE] Error after 100 iterations with early exaggeration: 0.691443
[t-SNE] Error after 150 iterations: 0.586644


In [69]:
df_tsne = pd.DataFrame()
df_tsne['x-tsne'] = tsne_results[:,0]
df_tsne['y-tsne'] = tsne_results[:,1]
df_tsne['label'] = labels

In [70]:
chart = ggplot( df_tsne, aes(x='x-tsne', y='y-tsne', color='label') ) \
        + geom_point(size=100) \
        + ggtitle("tSNE dimensions colored by cluster") \
        #+ scale_colour_gradient(low="coral", high="steelblue")
    
chart

<ggplot: (-9223371832304172856)>

## DBSCAN Clustering

In [71]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

### K-distance plot

In [122]:
def k_distances2(x, k):
    dim0 = x.shape[0]
    dim1 = x.shape[1]
    p=-2*x.dot(x.T)+np.sum(x**2, axis=1).T+ np.repeat(np.sum(x**2, axis=1),dim0,axis=0).reshape(dim0,dim0)
    p = np.sqrt(p)
    p.sort(axis=1)
    p=p[:,:k]
    pm= p.flatten()
    pm= np.sort(pm)
    return p, pm

In [131]:
tnse_values = df_tsne[['x-tsne', 'y-tsne']].values

In [132]:
#k = 2 ** data.shape[1]
k = math.floor(math.log(len(tnse_values)))
m, m2= k_distances2(tnse_values, k)
plt.plot(m2)
plt.ylabel("k-distances")
plt.grid(True)
plt.show()

C:\Users\cseve\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in sqrt


In [137]:
db = DBSCAN(eps=0.9, min_samples=k).fit(data)

In [138]:
np.unique(db.labels_)

array([0], dtype=int64)

In [135]:
df_tsne['label'] = db.labels_

In [136]:
chart = ggplot( df_tsne, aes(x='x-tsne', y='y-tsne', color='label') ) \
        + geom_point(size=100) \
        + ggtitle("tSNE dimensions colored by cluster") \
        #+ scale_colour_gradient(low="coral", high="steelblue")
    
chart

<ggplot: (-9223371832300787491)>